# ViT implementation from [lucidrains](https://github.com/lucidrains/vit-pytorch#vision-transformer---pytorch) explained

## Transformer Encoder
<img src="./visuals/transformer_encoder.png" alt="transformer_encoder" width="200"/>

#### Norm (here: 'PreNorm')

In [ ]:
from torch import nn

class PreNorm(nn.Module):
    def __init__(self, dim, attention_or_mlp):
        super().__init__()

        self.norm = nn.LayerNorm(dim)
        self.attention_or_mlp = attention_or_mlp

    def forward(self, x, **kwargs):
        normalised_input = self.norm(x)
        attention_or_mlp_output = self.attention_or_mlp(normalised_input, **kwargs)

        return attention_or_mlp_output

This 'PreNorm' class is used in two places
1. to calculate the attention with the norm as the input
2. to calculate the MLP with the previous output normalised

#### MLP (here: 'FeedForward')
- a two-layer neural network
- a GELU  and a dropout after the first layer
- a dropout after the second layer

**Gaussian Error Linear Unit (GELU)**
<img src="./visuals/gelu.png" alt="gelu" width="400"/>

- activation function used in the most recent Transformers

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

#### Multi-Head Scaled Dot-Product Attention (here: 'Attention')
<img src="./visuals/attention.png" alt="gelu" width="600"/>

In [ ]:
import torch
from torch import nn
from einops import rearrange, repeat

class Attention(nn.Module):
    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

**Q, K, V**
We want to get the linear embeddings of Q, K and V before passing them to the attention
- to get Q, K and V (or in the sketch - the putput of the linear layers after before the attention) we take the input (patch-embedding) and pass it through a linear layer with output dimension three times the inner dimension
    - inner dimension is the amount of heads times their dimensions
    - times three, because we will have to split it up into three
    - we could use separate networks for this, but like this you can combine them into one
    - this is done with 'qkv = self.to_qkv(x).chunk(3, dim=-1)'
- we need to do the rearranging, because we need to split the inner dimensions back into head and dim_head (reverse concatenation)

Then we multiply Q and K and scale it

Then we apply the softmax and a dropout

Then we multiply with V

Then we rearrange again (concatenation)

Finally we pass it through a linear layer and a dropout

#### Transformer Encoder (here: 'Transformer')
<img src="./visuals/transformer_encoder.png" alt="transformer_encoder" width="100"/>

In [ ]:
class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

- in 'init()' we create the multiple 'layers' of the transformer encoder
    - 'depth' corresponds to 'L' in the diagram
    - each layer consists of a multi-head attention (with the norm) and of an MLP (with the norm)
In 'forward()' we loop through all the layers and add them up
- first we calculate the attention and then pass the output to the MLP
- the output of the MLP is then fed into the attention in the next iteration

#### Vision Transformer

In [ ]:
from einops.layers.torch import Rearrange

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '1 n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        return self.mlp_head(x)

##### Patch-embedding

In [ ]:
img = np.zeros((1, 3, 224, 224))
patch_height = 32
patch_width = 32
patch_dim = 32
dim = 128 # hyperparameter
# (the '1' is the batch size, 3 is rgb, rest is image dim)
to_patch_embedding = nn.Sequential(
    Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_height, p2=patch_width),
    nn.Linear(patch_dim, dim),
        )

x = to_patch_embedding(img)

the embedding is a linear layer - projection of the patches (image)
- 'h' is the amount of patches vertically
- 'w' is the amount of patches horizontally
- so it gets rearranged into 'batch_size' 'amount of patches' 'RGB pixels per patch')
- and then passed through a linear layer
- patch_dim = p1 * p2 * c

##### CLS tokens

In [ ]:
# x has shape [1, 49, 128]

# create a token that will be added to each patch
cls_token = nn.Parameter(torch.randn(1, 1, dim))

# repeat the token for the amount of elements in the batch_size
cls_tokens = repeat(cls_token, '1 n d -> b n d', batch_size=1)

# add the embedding at the beginnign
x = torch.cat((cls_tokens, x), dim=1)

##### Position embedding

In [ ]:
num_patches = 7*7
n = num_patches
pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))

# the second dimension of the position embedding is one more than num_patches (because of the classification token)
x += pos_embedding[:, :(n + 1)]
# position embedding has dimension (1, 50, 28) and it gets added to each element of the batch